# 功能整合

类`ChangEDataManager`，包含以下功能：

- 本地数据文件夹设置、数据文件数统计、定制条件查询
- 远程数据查询和下载
- 载入pds文件并处理为图像数据（numpy？cv2？pyqt？）
- 将图像数据保存为PNG，可调缩放

还未考虑好的功能：

- 双目图像自动查找并返回图像数据
- 同批次图像自动查找并镶嵌拼合
- 全景图巡视

`ChangEDataManager`是用来在后端处理数据的类，前端的`ChangEViewer`将请求发来之后，`ChangEDataManager`会将处理之后的数据（字符串、列表或图像数据）返回给`ChangEViewer`

## 1. `ChangEDataManager`

由于嫦娥公开数据文件数量众多，文件名超长，所以这里专门设计了一个类`ChangEDataManager`，将数据的远程查询、下载、缓存、及本地标签查询等功能集成在一起，便于管理和访问。

### 1.1 query()

新建`ChangEDataManager`的时候需要配置本地数据缓存目录。

`query()`的流程：

1. 将先查询本地缓存，获得一个list；
2. 空查询远端服务器，获得符合条件的数据数目；
3. 比较本地和远端的数目，如果相同就不查询远端，如果不同，则进一步查询远端，获得远端list；
4. 两个list进行比较，标出区别，得出完整的lsit
5. 返回带标记的list

In [1]:
import os
import re
import requests
import time
import atool
from PIL import Image
import numpy as np
import pds4_tools
import colour
import colour_demosaicing
from skimage import exposure
import cv2

class ChangEDataManager:
    def __init__(self, path='./data'):
        """
        初始化内部变量，扫描数据文件夹，并现实初始化信息
        """
        self.__cache_path = path
        self.__data = []
        self.info()
        pass

    def scan_cache(self, with_hidden=False):
        """
        扫描数据文件夹，将文件名保存下来，并返回文件数量
        """        
        self.__data = []
        for f in os.listdir(self.__cache_path):
            if with_hidden == False and f[0] == '.':
                pass
            else:
                self.__data.append(f)
        count = len(self.__data)
        return count
    
    def info(self):
        print('嫦娥数据管理器 v0.2')
        print('数据文件目录 : %s' % (self.__cache_path))
        print('数据文件数目 ：%d' % (self.scan_cache()))
        #for f in os.listdir(self.__cache):
        #    print(f)
        pass
    
    def get_ChangE_download_link(self, pageNo, pageSize, task_id='CE4', data_type='SCI', payload='PCAM', data_grade='2B', date_start='default', date_end='today'):
        """
        通过requests.get()发起远程查询，说明详见“01下载数据”。尽量用query_remote()调用，而不是直接调用本函数。
        """
        # 默认情况下，根据任务设置查询的起始时间
        if date_start == 'default':
            if task_id == 'CE5':
                date_start = '2020-11-30 00:00:00'
            if task_id == 'CE4':
                date_start = '2019-01-02 00:00:00'
            if task_id == 'CE3':
                date_start = '2009-12-31 00:00:00'
            if task_id == 'CE2':
                date_start = '2010-10-01 00:00:00'
            if task_id == 'CE1':
                date_start = '2007-10-24 00:00:00'

        # 默认情况下，设置查询的结束时间为当天
        if date_end == 'today':
            date_end = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

        # 组装参数
        url = 'https://' + 'moon.bao.ac.cn' + '/proSearch!proSearchList.search'
        inputJsonString = '{"globalLang":null,"DM_DTYPE":"%s","DM_TASKID":"%s","DM_ZHNAME":"%s","DM_GRADE":"%s","DM_CBTIMEStart":"%s","DM_CBTIMEend":"%s","order_by":"desc","qsList":"","qsStart":"","qsEnd":"","bianhao":""}' % (data_type, task_id, payload, data_grade, date_start, date_end)
        param = {
            '_dc': '1615623500055', 
            'pageNo': str(pageNo), 
            'pageSize': str(pageSize), 
            'inputJsonString': inputJsonString, 
            'page': str(pageNo), 
            'start': str((pageNo-1)*pageSize), 
            'limit': str(pageSize)}

        # 访问网站，查询
        r = requests.get(url, param)
        if r.ok:
            link = []
            link.append(r.json()['totalCount'])
            for data in r.json()['returndata']:
                # print(atool.pretty(data))
                # 真实的下载链接
                link.append(data['dmStorepath'].replace('/PUBDATA/', 'http://moon.bao.ac.cn/cedownload/'))

            return link
        else:
            return None
    
    def is_datafile_exist(self, filename, filebuf=None, path='.', method='name'):
        """
        检查给定的文件是否已经存在
        目前只比对文件名
        """
        path = self.__cache_path
        if method == 'name':
            for f in os.listdir(path):
                if f[0] == '.':
                    pass
                else:
                    if filename == f:
                        return True
            return False
        elif method == 'size':
            pass
        elif method == 'content':
            pass

    def download(self, url, path):
        """
        将给定的url下载到path中
        """
        def pleasant_size(size):
            if size > 1024:
                size = size/1024
                [rs, unit] = pleasant_size(size)
                if unit == 'B':
                    return [rs, 'kB']
                elif unit == 'kB':
                    return [rs, 'MB']
                elif unit == 'MB':
                    return [rs, 'GB']
                elif unit == 'TB':
                    return [rs, 'TB']
                elif unit == 'PB':
                    return [rs, 'PB']
            else:
                return [size, 'B']
        
        path = self.__cache_path
        filename = url[(url.rfind('/')+1):len(url)]

        r = requests.get(url, stream=True)

        if r.ok:
            realsize = int(r.headers['Content-Length'])
            [size, unit] = pleasant_size(realsize)
            print('开始下载 %s（%.2f %s） ' % (filename, size, unit), end='')
            finished = 0
            with open(path + '/' + filename, "wb") as data:
                for chunk in r.iter_content(chunk_size=1024):
                    if chunk:
                        data.write(chunk)
                        finished = finished + len(chunk)
                    if (finished>realsize/10):
                        print('.', end='')
                        finished=0
            data.close()
            print('完成')
        else:
            print(r, filename, '连接失败')
    
    def query_and_download_all(self, task_id='CE4', data_type='SCI', payload='PCAM', data_grade='2B', date_start='default', date_end='today'):
        # 查第0页，不会返回具体数据，但是会返回数据文件的数目
        links = self.get_ChangE_download_link(0, 10, task_id, data_type, payload, data_grade, date_start, date_end)
        if links != None:
            # 获取数据总条目
            totalcount = int(links[0])
            print('查询到数据文件共 %d 个' % (totalcount))

            print('开始下载...')

            # 每页10条数据，反复“查询-下载”
            for i in range(int(totalcount/10+1)):
                print(i+1)
                links = self.get_ChangE_download_link(i+1, 10, task_id, data_type, payload, data_grade, date_start, date_end)
                if links != None:
                        for l in links[1:len(links)]:
                            filename = l[(l.rfind('/')+1):len(l)]
                            if self.is_datafile_exist(filename, path='data'):
                                print('文件 %s 已存在' % (filename))
                            else:
                                self.download(l, 'data')
                        print('下载完成')
                        time.sleep(0.5)
    
    def query_remote(self, qtask='CE4', qseq='*', qbatch='*', qcolor='*', qsite='*', qpayload='*', qdata_type='*', qdata_grade='*', qext='*'):
        """
        get_ChangE_download_link()函数的套壳
        """
        # 处理各参数
        task_id = '' if qtask == '*' else qtask
        data_type = '' if qdata_type == '*' else qdata_type
        payload = '' if qpayload == '*' else qpayload
        data_grade = '' if qdata_grade == '*' else qdata_grade
            
        # 先查第0页，获得数据总条目
        r = get_ChangE_download_link(0, 10, task_id, data_type, payload, data_grade, date_start, date_end)
        if r == None:
            return r
        # 获取数据总条目
        totalcount = int(r[0])
        print(totalcount)
        
        # TODO:
        # 多次查询将文件名和下载链接提取出来
        # 用其他的查询条件去除多余的结果
        # 返回列表
        pass
        
    def query_local(self, qtask='CE4', qseq='*', qbatch='*', qcolor='*', qsite='*', qpayload='*', qdata_type='*', qdata_grade='*', qext='*'):
        """
        发起一次本地查询，遍历self.__data中保存的所有文件名，返回符合查询条件的文件名
        """
        def check_tag(filename, tags):
            # 数据文件名样式有如下几种：
            # CE4_GRAS_PCAML-C-000_SCI_N_20200227111928_20200227111928_0121_B.2BL
            # 匹配样式：(.*)_(.*)_(.*)-(.*)-(.*)_(.*)_(.*)_([0-9]*)_([0-9]*)_([0-9]{4})_(.*)\.(.*)
            pattern = []
            pattern.append(r'(.*)_(.*)_(.*)-(.*)-(.*)_(.*)_(.*)_([0-9]*)_([0-9]*)_([0-9]{4})_(.*)\.(.*)')
            # CE4_GRAS_ASAN-SCI_SCI_N_20190111150201_20190112102100_0003_B.2B
            # 匹配样式：(.*)_(.*)_(.*)-(.*)_(.*)_(.*)_([0-9]*)_([0-9]*)_([0-9]{4})_(.*)\.(.*)
            pattern.append(r'(.*)_(.*)_(.*)-(.*)_(.*)_(.*)_([0-9]*)_([0-9]*)_([0-9]{4})_(.*)\.(.*)')
            
            match_obj = re.match(pattern[0], filename)
            match_obj2= re.match(pattern[1], filename)
            # 提取出文件名中的各种标签
            if match_obj != None:
                task = match_obj.group(1) # 任务
                site = match_obj.group(2) # 站点
                payload = match_obj.group(3) # 载荷，如果是PCAM的话，还需要指出是左眼还是右眼
                color = match_obj.group(4) # C、Q
                seq = match_obj.group(5) # 序号
                data_type = match_obj.group(6) # 数据类型
                unknown = match_obj.group(7) # 未知
                start_time = match_obj.group(8) # 起时间
                end_time = match_obj.group(9) # 止时间
                batch = match_obj.group(10) # 序号？
                data_grade = match_obj.group(11) # 数据等级
                ext = match_obj.group(12) # 扩展名
            elif match_obj2 != None:
                task = match_obj2.group(1) # 任务
                site = match_obj2.group(2) # 站点
                payload = match_obj2.group(3) # 载荷，如果是PCAM的话，还需要指出是左眼还是右眼
                color = 'N/A' # C、Q
                seq = '-1' # 序号
                data_type = match_obj2.group(5) # 数据类型
                unknown = match_obj2.group(6) # 未知
                start_time = match_obj2.group(7) # 起时间
                end_time = match_obj2.group(8) # 止时间
                batch = match_obj2.group(9) # 序号？
                data_grade = match_obj2.group(10) # 数据等级
                ext = match_obj2.group(11) # 扩展名
            else:
                return None
                
            ftags = [task, seq, batch, color, site, payload, data_type, data_grade, ext]
            #print(ftags)
                
            # 逐个检查标签是否匹配
            checklist = []
            for i in range(len(tags)):
                t = tags[i]
                f = ftags[i]
                if t == '*':
                    checklist.append(True)
                elif t == f:
                    checklist.append(True)
                else:
                    checklist.append(False)
            return checklist
        
        result = []

        # 依次判断文件名是否满足查询
        for f in self.__data:
            checked = True
            checklist = check_tag(f, [qtask, qseq, qbatch, qcolor, qsite, qpayload, qdata_type, qdata_grade, qext])
            #print(f, checklist)
            if checklist == None:
                print('注意：文件名格式无法匹配 %s' % (f))
            else:
                for c in checklist:
                    if c == False:
                        checked = False
                        break
                if checked == True:
                    result.append(f)
        
        return sorted(result)
    
    def query(self, qtask='CE4', qseq='*', qbatch='*', qcolor='*', qsite='*', qpayload='*', qdata_type='*', qdata_grade='*', qext='*'):
        """
        统一查询。先查本地，再查远程。本地标注已缓存，远程标注下载链接
        """
        local_list = self.query_local(qtask, qseq, qbatch, qcolor, qsite, qpayload, qdata_type, qdata_grade, qext)
        remote_list= self.query_remote(qtask, qseq, qbatch, qcolor, qsite, qpayload, qdata_type, qdata_grade, qext)
        query_list=[]
        for f in remote_list:
            if f in local_list:
                query_list.append(['c', f])
            else:
                query_list.append(['download_url', f])
        print(atool.pretty(query_list))
        
    def load_2A2B(self, filename):
        """
        读入2A或2B数据，并返回处理过的、原始大小的numpy数组
        """
        img = {}

        # 读入嫦娥4号2BL数据
        img['pds_data'] = pds4_tools.read(filename)

        # 将图像数据提取出来
        img['raw_data'] = np.asanyarray(img['pds_data'][0].data)
        img['raw_data'] = img['raw_data'] / 1023      #10位的图像数据归一化
        print('提取原始数据并归一化', img['raw_data'].shape, img['raw_data'].ndim)

        # de-bayer
        rgb_data = colour.cctf_encoding(colour_demosaicing.demosaicing_CFA_Bayer_bilinear(img['raw_data'], 'RGGB')) 
        img['rgb_data'] = rgb_data
        print('CFA上色（de-bayer）', rgb_data.shape, rgb_data.ndim)

        # 直方图拉伸
        lower, upper = np.percentile(img['rgb_data'], (5,99))
        img['intensity_data'] = exposure.rescale_intensity(img['rgb_data'], in_range=(lower, upper)) 
        print('直方图拉伸', lower, upper)

        return img['intensity_data']


    def convert_pds_2_png(self, filename, scale=1.0, input_path='data', output_path='png'):
        """
        filename是包含路径的文件名
        """
        img = {}
        input_path = self.__cache_path

        img['intensity_data'] = self.load_2A2B(filename)
        print(type(img['intensity_data']))

        # 缩放
        if scale != 1.0:
            print('图像缩放', scale)
            height, width = img['intensity_data'].shape[:2]
            size = (int(width*scale), int(height*scale))  
            img['scaled_data'] = cv2.resize(img['intensity_data'], size, interpolation=cv2.INTER_AREA)
        else:
            img['scaled_data'] = img['intensity_data']

        # 保存到文件
        p = img['scaled_data'] * 255
        p = p.astype('uint8')
        output_filename = filename.replace(input_path, output_path) + '.png'
        cv2.imwrite(output_filename, cv2.cvtColor(p, cv2.COLOR_RGB2BGR)) # opencv的红色和蓝色会对调，需要改变一下颜色顺序
        print('保存到文件', output_filename)
    
    def convert_pds_from_list(self, filelist, scale=1.0, input_path='data', output_path='png'):
        input_path = self.__cache_path
        for filename in filelist:
            self.convert_pds_2_png(input_path + '/' + filename, scale, input_path, output_path) # 可设置输入目录与输出目录
            
    def convert_SBS_png(self, leftfile, rightfile):
        # 读入左右目图片
        leftimg0 = self.load_2A2B(leftfile)
        rightimg0= self.load_2A2B(rightfile)
        # 变形并调整尺寸
        size = (960, 1080)  
        leftimg1 = cv2.resize(leftimg0, size, interpolation=cv2.INTER_AREA)
        rightimg1= cv2.resize(rightimg0, size, interpolation=cv2.INTER_AREA)
        # 将两个图像合并为一
        img = np.hstack((leftimg1, rightimg1)) 
        return img

    def convert_SBS_from_list(self, leftlist, rightlist, inputpath, outputpath):
        inputpath = self.__cache_path
        for i in range(len(leftlist)):
            leftfile = inputpath + '/' + leftlist[i]
            rightfile= inputpath + '/' + rightlist[i]
            img = self.convert_SBS_png(leftfile, rightfile)
            p = img * 255
            p = p.astype('uint8')
            cv2.imwrite(outputpath+'/'+leftlist[i].replace('PCAML', 'PCAMLR')+'.png', cv2.cvtColor(p, cv2.COLOR_RGB2BGR))

In [2]:
cedm = ChangEDataManager('data')

嫦娥数据管理器 v0.2
数据文件目录 : data
数据文件数目 ：20640


In [19]:
list = cedm.get_ChangE_download_link(10, 100, task_id='CE3', payload='', data_grade='', date_start='')
print(atool.pretty(list))

[
    254138
    http://moon.bao.ac.cn/cedownload/MUVT/2A/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230809_20150110230809_0393_B.2A
    http://moon.bao.ac.cn/cedownload/MUVT/2B/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230809_20150110230809_0393_B.2B
    http://moon.bao.ac.cn/cedownload/MUVT/2A/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230604_20150110230604_0393_B.2A
    http://moon.bao.ac.cn/cedownload/MUVT/2B/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230604_20150110230604_0393_B.2B
    http://moon.bao.ac.cn/cedownload/MUVT/2A/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230358_20150110230358_0393_B.2A
    http://moon.bao.ac.cn/cedownload/MUVT/2B/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230358_20150110230358_0393_B.2B
    http://moon.bao.ac.cn/cedownload/MUVT/2A/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230153_20150110230153_0393_B.2A
    http://moon.bao.ac.cn/cedownload/MUVT/2B/CE3_BMYK_MUVT-P-0099_SCI_N_20150110230153_20150110230153_0393_B.2B
    http://moon.bao.ac.cn/cedownload/MUVT/2A/CE3_BMYK_MUVT-P-0099_SCI_N_20150110225947_2015

In [11]:
cedm.query_and_download_all(task_id='CE4', payload='PCAML', data_grade='', date_start='20200401')

查询到数据文件共 220 个
开始下载...
1
文件 CE4_GRAS_PCAML-C-011_SCI_N_20200428095013_20200428095013_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-011_SCI_N_20200428095013_20200428095013_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-010_SCI_N_20200428094844_20200428094844_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-010_SCI_N_20200428094844_20200428094844_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20200428094715_20200428094715_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20200428094715_20200428094715_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-008_SCI_N_20200428094546_20200428094546_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-008_SCI_N_20200428094546_20200428094546_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-007_SCI_N_20200428094417_20200428094417_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-007_SCI_N_20200428094417_20200428094417_0128_B.2BL 已存在
下载完成
2
文件 CE4_GRAS_PCAML-C-006_SCI_N_20200428094248_20200428094248_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-006_SCI_N_20200428094248_20200428094248_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-005_SCI_N_20200428093658_20200428093658_0128_B.2B 已存在


12
文件 CE4_GRAS_PCAML-Q-054_SCI_N_20200419101245_20200419101245_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-054_SCI_N_20200419101245_20200419101245_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-053_SCI_N_20200419101125_20200419101125_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-053_SCI_N_20200419101125_20200419101125_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-052_SCI_N_20200419101005_20200419101005_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-052_SCI_N_20200419101005_20200419101005_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-051_SCI_N_20200419100845_20200419100845_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-051_SCI_N_20200419100845_20200419100845_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-050_SCI_N_20200419100725_20200419100725_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-050_SCI_N_20200419100725_20200419100725_0126_B.2BL 已存在
下载完成
13
文件 CE4_GRAS_PCAML-Q-049_SCI_N_20200419100605_20200419100605_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-049_SCI_N_20200419100605_20200419100605_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-048_SCI_N_20200419100445_20200419100445_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-0

23
下载完成


In [29]:
q = cedm.query_local(qtask='CE4', qpayload='PCAML', qcolor='C', qbatch='*', qseq='*', qext='2BL')
print('查询命中数量：%d\n' % (len(q)))
print(atool.pretty(q))


查询命中数量：290

[
    CE4_GRAS_PCAML-C-000_SCI_N_20190104041518_20190104041518_0001_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190110211129_20190110211129_0002_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190112100836_20190112100836_0003_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190208204547_20190208204547_0007_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190228081519_20190228081519_0011_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190429143738_20190429143738_0033_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190508023258_20190508023258_0035_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190510100115_20190510100115_0038_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190708043358_20190708043358_0048_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190729011859_20190729011859_0050_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190806113452_20190806113452_0053_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190826032641_20190826032641_0056_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190827051956_20190827051956_0059_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190827094241_2019

In [28]:
q4 = cedm.query_local(qtask='CE4', qpayload='PCAML', qcolor='C', qbatch='0007', qseq='*', qext='2BL')
q1 = cedm.query_local(qtask='CE4', qpayload='PCAML', qcolor='C', qbatch='0033', qseq='*', qext='2BL')
q2 = cedm.query_local(qtask='CE4', qpayload='PCAML', qcolor='C', qbatch='0035', qseq='*', qext='2BL')
q3 = cedm.query_local(qtask='CE4', qpayload='PCAML', qcolor='C', qbatch='0038', qseq='*', qext='2BL')
leftlist = q4+q1+q2+q3
print('查询命中数量：%d\n' % (len(leftlist)))
print(atool.pretty(leftlist))

q4 = cedm.query_local(qtask='CE4', qpayload='PCAMR', qcolor='C', qbatch='0007', qseq='*', qext='2BL')
q1 = cedm.query_local(qtask='CE4', qpayload='PCAMR', qcolor='C', qbatch='0033', qseq='*', qext='2BL')
q2 = cedm.query_local(qtask='CE4', qpayload='PCAMR', qcolor='C', qbatch='0035', qseq='*', qext='2BL')
q3 = cedm.query_local(qtask='CE4', qpayload='PCAMR', qcolor='C', qbatch='0038', qseq='*', qext='2BL')
rightlist = q4+q1+q2+q3
print('查询命中数量：%d\n' % (len(rightlist)))
print(atool.pretty(rightlist))

查询命中数量：18

[
    CE4_GRAS_PCAML-C-000_SCI_N_20190208204547_20190208204547_0007_B.2BL
    CE4_GRAS_PCAML-C-001_SCI_N_20190208204706_20190208204706_0007_B.2BL
    CE4_GRAS_PCAML-C-002_SCI_N_20190208204825_20190208204825_0007_B.2BL
    CE4_GRAS_PCAML-C-000_SCI_N_20190429143738_20190429143738_0033_B.2BL
    CE4_GRAS_PCAML-C-001_SCI_N_20190429143857_20190429143857_0033_B.2BL
    CE4_GRAS_PCAML-C-002_SCI_N_20190429144016_20190429144016_0033_B.2BL
    CE4_GRAS_PCAML-C-003_SCI_N_20190429144135_20190429144135_0033_B.2BL
    CE4_GRAS_PCAML-C-004_SCI_N_20190429144254_20190429144254_0033_B.2BL
    CE4_GRAS_PCAML-C-005_SCI_N_20190429144413_20190429144413_0033_B.2BL
    CE4_GRAS_PCAML-C-006_SCI_N_20190429144958_20190429144958_0033_B.2BL
    CE4_GRAS_PCAML-C-007_SCI_N_20190429145117_20190429145117_0033_B.2BL
    CE4_GRAS_PCAML-C-008_SCI_N_20190429145236_20190429145236_0033_B.2BL
    CE4_GRAS_PCAML-C-009_SCI_N_20190429145355_20190429145355_0033_B.2BL
    CE4_GRAS_PCAML-C-010_SCI_N_20190429145514_20190

In [27]:
%%time
cedm.convert_SBS_from_list(leftlist, rightlist, 'data', 'png')

Processing label: data/CE4_GRAS_PCAML-C-000_SCI_N_20190429143738_20190429143738_0033_B.2BL
2021-05-19 00:53:43 core.py [line:189] INFO: Processing label: data/CE4_GRAS_PCAML-C-000_SCI_N_20190429143738_20190429143738_0033_B.2BL

Now processing a Array_2D_Image structure: ARRAY_0
2021-05-19 00:53:43 core.py [line:304] INFO: Now processing a Array_2D_Image structure: ARRAY_0

提取原始数据并归一化 (1728, 2352) 2
CFA上色（de-bayer） (1728, 2352, 3) 3
直方图拉伸 0.348695910986 0.805331361145
Processing label: data/CE4_GRAS_PCAMR-C-000_SCI_N_20190429143722_20190429143722_0033_B.2BL
2021-05-19 00:53:44 core.py [line:189] INFO: Processing label: data/CE4_GRAS_PCAMR-C-000_SCI_N_20190429143722_20190429143722_0033_B.2BL

Now processing a Array_2D_Image structure: ARRAY_0
2021-05-19 00:53:45 core.py [line:304] INFO: Now processing a Array_2D_Image structure: ARRAY_0

提取原始数据并归一化 (1728, 2352) 2
CFA上色（de-bayer） (1728, 2352, 3) 3
直方图拉伸 0.328182198208 0.80304019261
Processing label: data/CE4_GRAS_PCAML-C-001_SCI_N_2019042

Now processing a Array_2D_Image structure: ARRAY_0
2021-05-19 00:54:05 core.py [line:304] INFO: Now processing a Array_2D_Image structure: ARRAY_0

提取原始数据并归一化 (1728, 2352) 2
CFA上色（de-bayer） (1728, 2352, 3) 3
直方图拉伸 0.450518203773 0.766369939398
Processing label: data/CE4_GRAS_PCAML-C-009_SCI_N_20190429145355_20190429145355_0033_B.2BL
2021-05-19 00:54:06 core.py [line:189] INFO: Processing label: data/CE4_GRAS_PCAML-C-009_SCI_N_20190429145355_20190429145355_0033_B.2BL

Now processing a Array_2D_Image structure: ARRAY_0
2021-05-19 00:54:06 core.py [line:304] INFO: Now processing a Array_2D_Image structure: ARRAY_0

提取原始数据并归一化 (1728, 2352) 2
CFA上色（de-bayer） (1728, 2352, 3) 3
直方图拉伸 0.41675741831 0.799731553274
Processing label: data/CE4_GRAS_PCAMR-C-009_SCI_N_20190429145339_20190429145339_0033_B.2BL
2021-05-19 00:54:08 core.py [line:189] INFO: Processing label: data/CE4_GRAS_PCAMR-C-009_SCI_N_20190429145339_20190429145339_0033_B.2BL

Now processing a Array_2D_Image structure: ARRAY_0
2021-0